In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

class FNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(3 * 32 * 32, 500), # Fully connected layer from input to 500 neurons
            nn.ReLU(),
            nn.Linear(500, num_classes)
        )

    def forward(self, x):
        x = torch.flatten(x, 1)
        return self.layers(x)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize the images
])

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform) #Load dataset


'''
Split dataset into training and validation
'''

train_size = int(0.8 * len(dataset))  # Determine size of training set (80% of total)
val_size = len(dataset) - train_size # The remaining for validation
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

model = FNN(num_classes=10)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

'''
Function to train the model
'''
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # Set the model to training mode
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X) # Make predictions
        y_one_hot = nn.functional.one_hot(y, num_classes=10).float()
        loss = loss_fn(pred, y_one_hot)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

'''
Function to evaluate the model
'''
def validate(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad(): # Disable gradient computation
        for X, y in dataloader:
            pred = model(X) # Make predictions
            y_one_hot = nn.functional.one_hot(y, num_classes=10).float()
            val_loss += loss_fn(pred, y_one_hot).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= num_batches
    accuracy = correct / size
    print(f"Validation Error: \n Avg loss: {val_loss:>8f} \n Accuracy: {accuracy:>8f}")


epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    validate(val_loader, model, loss_fn)
print("Training is finished")

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64)

validate(test_loader, model, loss_fn) # test



Files already downloaded and verified
Epoch 1
-------------------------------
loss: 0.120646  [    0/40000]
loss: 0.091608  [ 6400/40000]
loss: 0.094523  [12800/40000]
loss: 0.089279  [19200/40000]
loss: 0.075868  [25600/40000]
loss: 0.074658  [32000/40000]
loss: 0.081729  [38400/40000]
Validation Error: 
 Avg loss: 0.076392 
 Accuracy: 0.417300
Epoch 2
-------------------------------
loss: 0.073993  [    0/40000]
loss: 0.066539  [ 6400/40000]
loss: 0.073642  [12800/40000]
loss: 0.074981  [19200/40000]
loss: 0.066513  [25600/40000]
loss: 0.078701  [32000/40000]
loss: 0.077454  [38400/40000]
Validation Error: 
 Avg loss: 0.073661 
 Accuracy: 0.460400
Epoch 3
-------------------------------
loss: 0.069961  [    0/40000]
loss: 0.076518  [ 6400/40000]
loss: 0.075417  [12800/40000]
loss: 0.070924  [19200/40000]
loss: 0.073214  [25600/40000]
loss: 0.079693  [32000/40000]
loss: 0.073551  [38400/40000]
Validation Error: 
 Avg loss: 0.074799 
 Accuracy: 0.460200
Epoch 4
------------------------